# Item Collaborative Filtering
* See `ItemCollaborativeFilteringBase.ipynb` for algorithm details
* The weights here are the cosine correlation between the two items

In [1]:
name = "ItemCF";

In [2]:
using NBInclude
@nbinclude("ItemCFBase.ipynb");

## Compute cosine correlations

In [3]:
function get_correlation_matrix_outdir(residual_alphas)
    # if the matrix is already stored on disk, return its filepath
    # otherwise, regenerate the matrix and store it to disk
    outdir = "$name/$(hash(residual_alphas))"
    if ispath("../../data/alphas/$outdir")
        return outdir
    end

    @debug "generating similarity matrix for $residual_alphas"    
    training = get_residuals("training", residual_alphas)
    R = sparse(
        training.user,
        training.item,
        training.rating,
        maximum(training.user),
        maximum(training.item),
    )
    S = zeros(maximum(training.item), maximum(training.item))

    norms = map(norm, eachslice(R, dims = 2))
    norms[norms.==0] .= 1 # prevent division by 0
    @tprogress Threads.@threads for i = 1:size(S)[1]
        S[:, i] = vec(R[:, i]' * R) ./ norms ./ norms[i]
    end

    S = convert.(Float32, S)    
    write_params(Dict("S" => S), outdir = outdir)
    outdir
end;

## Setup hyperparameters

In [4]:
downcast_to_int(x) = isinteger(x) ? Int(x) : x
item_cf_params = [[
        cf_params(
            name = "ItemCF.$K",
            training_residuals = ["UserItemBiases"],
            validation_residuals = ["UserItemBiases"],
            neighborhood_type = "abs",
            S = get_correlation_matrix_outdir(["UserItemBiases"]),
            K = K,
            λ = [1.0, 1.0, 0.0],
        ) for K in downcast_to_int.([2^4, 2^6, 2^8, 2^10])
    ];
];

## Train models

In [ ]:
for param in item_cf_params
    optimize_model(param)
end

Progress:  12%|███▎                       |  ETA: 0:05:33 (23.89 ms/it)